In [1]:
from bs4 import BeautifulSoup
from utils import get_chromedriver
import requests
from soup2dict import convert
import pandas as pd
import numpy as np
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import config

In [2]:
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\0 Tools\Download ABS ranking\WebDriver\chromedriver"

In [3]:
URL = 'https://charteredabs.org/academic-journal-guide-2021-view/'

In [4]:
SAVE_PATH = './/ABS list.xlsx'

In [5]:
driver=get_chromedriver(chromedriver_path=CHROMEDRIVER_PATH)
driver.get(URL)

wait=WebDriverWait(driver, 20)

# login
print('Login... ', end='')
login_button=wait.until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/div/div[2]/p/a')))
login_button.click()

user=wait.until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/form/p[1]/input")))
user.send_keys(config.User)
pwd=wait.until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/form/div/div/input")))
pwd.send_keys(config.Password)

login_button=wait.until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/form/p[3]/input[1]')))
login_button.click()
print('Done')

driver.get(URL)

# select All
print('Show All... ', end='')
showButton=wait.until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[4]/div/div[2]/div[1]/div/div/div/div[2]/div[2]/div[3]/label/div/button")))
actions=ActionChains(driver)
actions.move_to_element(showButton).pause(2).click().perform()
AllButton=wait.until(
    EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/div/div[2]/div[1]/div/div/div/div[2]/div[2]/div[3]/label/div/div/ul/li[7]/a')))
AllButton.click()
# wait for the page to load
while True:
    page_source=BeautifulSoup(driver.page_source, 'html.parser').prettify()
    res = re.search(r',\d{3}\sentries', page_source)
    span=res.span()
    total_expected_entries=page_source[(span[0]-4):span[1]].replace(',', '')
    total_expected_entries=int(''.join([x for x in total_expected_entries if x.isdigit()]))
    total_expected_entries_string='{:,}'.format(total_expected_entries)
    string_to_appear=f'Showing 1 to {total_expected_entries_string} of {total_expected_entries_string} entries'
    if string_to_appear in page_source:
        break        
print('Done')

# scrape list
soup = convert(BeautifulSoup(driver.page_source, 'html.parser'))
table=soup['html'][0]['body'][0]['div'][0]['div'][3]['div'][0]['div'][1]['div'][0]['div'][0]['div'][0]['div'][0]['div'][1]['div'][1]['table'][0]

col_names=[]
for tt in table['thead'][0]['tr'][1]['th']:
    col_names.append(tt['navigablestring'][0])
    
df_list=pd.DataFrame()
for tt in table['tbody'][0]['tr']:
    issn=tt['td'][0]['navigablestring'][0]
    field=tt['td'][1]['navigablestring'][0]
    title=tt['td'][2]['navigablestring'][0]
    publisher=field=tt['td'][3]['navigablestring'][0]
    scopus_link=''
    if 'a' in tt['td'][4].keys():
        scopus_link=tt['td'][4]['a'][0]['@href']
    wos_link=''
    if 'a' in tt['td'][5].keys():
        wos_link=tt['td'][5]['a'][0]['@href']
    rankings=[]
    for i in range(6, len(col_names)):
        rankings.append(tt['td'][i]['#text'])
    add_row=pd.DataFrame(np.array([issn, field, title, publisher, scopus_link, wos_link] + rankings).reshape(1,-1), columns=col_names)
    df_list=pd.concat([df_list, add_row])
print(f'\n\n{len(df_list)} journals found')
if total_expected_entries != len(df_list):
    print(f'####### Mismatch in downloaded journals: {total_expected_entries} expected.')
df_list.reset_index(drop=True).to_excel(SAVE_PATH, sheet_name='ABS list', index=False)
print(f'\n\nData exported in {SAVE_PATH}')

driver.close()

Login... Done
Show All... Done


1698 journals found


Data exported in .//ABS list.xlsx
